In [1]:
#path_name_documents = './Databases/prova/prova.jsonl'
#path_name_documents = './Databases/prova/prova4.jsonl'
path_name_documents = './Databases/prova/prova10.jsonl'
#path_name_documents = './Databases/prova/prova50.jsonl'
#path_name_documents = './Databases/prova/prova2000.jsonl'
#path_name_documents = './Databases/prova/prova30000.jsonl'


In [2]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time:.5f} seconds")
        return result
    return wrapper

In [3]:
import json
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


stop_words = set(stopwords.words('english'))

def stemmingLemming(filtered_tokens):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming or lemmatization on filtered tokens
    
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return filtered_tokens
    
 
    

def tokenize(path_name):
    
    with open(path_name, "r") as f:
        data = f.readlines()

        # Create an empty list to store the tokenized documents
        tokenized_docs = []

        # Loop through each line in the JSONL file
        for line in data:
            # Parse the JSON string into a Python dictionary
            doc = json.loads(line)

            # Extract the text from the dictionary
            text = doc['text']
            text = text.lower()  # Convert to lowercase
            #text = re.sub(r'\d+', '', text)  # Remove all numbers
            text = text.translate(str.maketrans('', '', string.punctuation))  # Remove all punctuation

            # Tokenize the text using NLTK
            tokens = word_tokenize(text)
            tokensStemLem = stemmingLemming(tokens)

            # Add the tokenized document to the list
            tokenized_docs.append(tokensStemLem)

        # Print the tokenized documents
    return tokenized_docs


tokenized_docs = tokenize(path_name_documents)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer


def calculateTFIDF(tokenized_docs):
    
    vectorizer = TfidfVectorizer()
    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names_out()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
    
        

tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(tokenized_docs)

# DA QUI PARALLEL SPARK

### UNA RIGA

In [5]:
def get_row_values_tfidf(tfidf_matrix, doc_id):
    num_docs = tfidf_matrix.shape[0]
    # Estrai la riga specificata dalla matrice TF-IDF come un array densamente popolato
    row_array = tfidf_matrix.toarray()[doc_id]
    # Creazione della lista di coppie chiave-valore
    row_values = [(index, value) for (index, value) in enumerate(row_array) if value != 0.0]
    # Restituisci la lista di coppie chiave-valore
    return doc_id, row_values,num_docs  


#doc_id, row_values, num_docs  = get_row_values_tfidf(tfidf_matrix_docs,0)

### TUTTE LE RIGHE

In [6]:
def get_all_row_values_tfidf(tfidf_matrix):
    num_docs = tfidf_matrix.shape[0]
    row_values_list = []
    for doc_id in range(num_docs):
        row_array = tfidf_matrix.toarray()[doc_id]
        row_values = [(index, value) for (index, value) in enumerate(row_array) if value != 0.0]
        row_values_list.append((doc_id, row_values))
    return row_values_list


In [7]:
import os
os.environ['PYSPARK_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'


### Funzione funzionante per python

## Implementazioni per Spark 

### Functions

In [8]:
from sklearn.metrics.pairwise import cosine_similarity


def sim(list1, list2):
    # Extract the values from the input lists
    values1 = [x for x, _ in list1]
    values2 = [x for x, _ in list2]
    
    # Pad the shorter list with zeros to match the dimensionality
    max_length = max(len(values1), len(values2))
    values1 += [0] * (max_length - len(values1))
    values2 += [0] * (max_length - len(values2))
    
    # Calculate the cosine similarity between the two lists
    similarity_matrix = cosine_similarity([values1], [values2])
    similarity = similarity_matrix[0][0]
    
    return similarity



# Funzione di mapping
def my_map(doc_id, document,threshold):
    map_results = []
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results


# Funzione di riduzione
def my_reduce(t, doc_list, threshold):
    emitted_results = []
    for i, doc_item1 in enumerate(doc_list):
        id1, d1 = doc_item1
        for j in range(i + 1, len(doc_list)):
            doc_item2 = doc_list[j]
            id2, d2 = doc_item2
            similarity = sim(d1, d2)
            if similarity >= threshold:
                emitted_results.append((id1+1, id2+1, similarity))
    return emitted_results


In [ ]:
def my_map1(doc_id, document, threshold):
    map_results = []
    
    def b(document):
        # Funzione ausiliaria per calcolare l'indice i più grande
        # per cui la similarità è sotto la soglia di similarità desiderata
        max_similarity = 0.0
        max_index = 0
        
        for i in range(1, len(document) + 1):
            subarray = document[:i]
            similarity = calculate_similarity(subarray, vmax[:i])  # Sostituisci con il calcolo effettivo della similarità
            
            if similarity <= threshold:
                max_similarity = similarity
                max_index = i
            else:
                break
        
        return max_index
    
    sorted_document = sorted(document.items())
    vmax = [value for _, value in sorted_document]
    
    for term, value in sorted_document:
        if value > threshold:
            map_results.append((term, [doc_id, document]))
    
    return map_results


### Execution with spark

In [9]:
datas = get_all_row_values_tfidf(tfidf_matrix_docs)

In [12]:
from pyspark import SparkConf, SparkContext
import time

start_time = time.time()
# Configura Spark per utilizzare 6 cores e 10 GB di RAM
conf = SparkConf().setAppName("MyApp").setMaster("local[8]").set("spark.executor.memory", "12g")
sc = SparkContext(conf=conf)

threshold = 0.3
# Dati di input



# Crea l'RDD di input
input_rdd = sc.parallelize(datas)
#print(input_rdd.collect())

print("MAP")
# Esegui la funzione di mapping
mapped_rdd = input_rdd.flatMap(lambda x: my_map(x[0], x[1], threshold))
#print(mapped_rdd.collect())

print("REDUCE")
# Esegui la funzione di riduzione
reduced_rdd = mapped_rdd.groupByKey().flatMap(lambda x: my_reduce([0], list(x[1]), threshold))



end_time = time.time()
total_time = end_time - start_time
print("Time of execution",total_time)
print("RESULTS")
# Raccogli i risultati
results = reduced_rdd.collect()
#print(results)
# Elimina duplicati
results = list(set(results))
#print(results)
for result in sorted(results):
    print(result)

# Ferma lo SparkContext
sc.stop()


MAP
REDUCE
Time of execution 0.15300297737121582
RESULTS
(1, 43, 0.4036821730697277)
(11, 31, 0.6456972907740861)
(13, 20, 0.3672149017514845)
(23, 39, 0.662805732685481)
(27, 36, 0.8785909672237726)
(30, 36, 0.9041935483968163)
(32, 34, 0.661304032747174)
(32, 49, 0.9206874495287574)
(34, 49, 0.7254599281514812)


In [11]:
sc.stop()